Model Deployment
---

In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
# READ THE DATA
df = pd.read_csv("data-week3.csv")

In [ ]:
numeric_cols = ['tenure', 'monthlycharges', 'totalcharges']
categorical_cols = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice','onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport','streamingtv', 'streamingmovies', 'contract', 'paperlessbilling','paymentmethod']

In [ ]:
# first split the data into ratio 80:20 for training and test dataset
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Shape
print("df_train shape::", df_full_train.shape)
print("df_test shape::", df_test.shape)

In [ ]:
# Reset the index
df_full_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [ ]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical_cols + numeric_cols].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train  = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model


In [ ]:
def predict(df, dv, model):
    dicts = df[categorical_cols + numeric_cols].to_dict(orient='records')

    X = dv.transform(dicts)
    y_preds = model.predict_proba(X)[:, 1]

    return y_preds

In [ ]:
C = 1.0
n_splits = 5

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = [] 

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_preds = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_preds)
    scores.append(auc)

print("C=%s %.3f +- %.3f" % (C, np.mean(scores), np.std(scores)))


In [ ]:
scores

In [ ]:
y_test = df_test.churn.values
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

In [ ]:
# Save the model
import pickle

In [ ]:
output_file = "model_C=%s.bin" %C
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [1]:
# Load the model
import pickle

In [2]:
# open pickle file in namespace
input_file = "model_C=1.0.bin"
with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [4]:
customer = {
    'gender':'female',
    'seniorcitizen':0,
    'partern':'yes',
    'dependents':'no', 
    'phoneservice':'no',
    'multiplelines':'no_phone_service', 
    'internetservice':'dsl',
    'onlinesecurity':'no',
    'onlinebackup':'yes',
    'deviceprotection':'no',
    'techsupport':'no',
    'streamingtv':'no',
    'streamingmovies':'no',
    'contract':'month-to-month',
    'paperlessbilling':'yes',
    'paymentmethod':'electronic_check',
    'tenure':1,
    'monthlycharges':29.85,
    'totalcharges':29.85
}

In [5]:
X = dv.transform([customer])

In [6]:
model.predict_proba(X)[0, 1]

0.6611993631365279